# Gridworld Reinforcement Learning

## Problem Description
- 4x4 Gridworld with states {0 ... 15}
- Terminal State = 9
- Actions = Up, Down, Left, Right
- Reward = -1 per step
- Discount Factor = 0.8
- Initial Value = -1 for all states except terminal

This notebook implements:
1. Iterative Policy Evaluation
2. Greedy Policy Extraction
3. Value Iteration


## Import Libraries

In [1]:
import numpy as np

## Environment Setup

In [2]:
grid_size = 4
gamma = 0.8
terminal_state = 9
actions = ['up','down','left','right']
num_states = grid_size * grid_size

def next_state(state, action):
    row, col = divmod(state, grid_size)

    if action == 'up':
        row = max(row - 1, 0)
    elif action == 'down':
        row = min(row + 1, grid_size - 1)
    elif action == 'left':
        col = max(col - 1, 0)
    elif action == 'right':
        col = min(col + 1, grid_size - 1)

    return row * grid_size + col

## Initialize Value Function

In [3]:
V = -1 * np.ones(num_states)
V[terminal_state] = 0

## 1. Iterative Policy Evaluation (Equiprobable Random Policy)

In [4]:
def policy_evaluation(V, iterations=3):
    for k in range(iterations):
        new_V = V.copy()

        for s in range(num_states):
            if s == terminal_state:
                continue

            value = 0
            for a in actions:
                ns = next_state(s, a)
                reward = -1
                value += 0.25 * (reward + gamma * V[ns])

            new_V[s] = value

        V = new_V
        print(f'Iteration {k+1}')
        print(V.reshape(grid_size, grid_size))

    return V

V_policy = policy_evaluation(V.copy(), 3)

Iteration 1
[[-1.8 -1.8 -1.8 -1.8]
 [-1.8 -1.6 -1.8 -1.8]
 [-1.6  0.  -1.6 -1.8]
 [-1.8 -1.6 -1.8 -1.8]]
Iteration 2
[[-2.44 -2.4  -2.44 -2.44]
 [-2.36 -2.08 -2.36 -2.44]
 [-2.04  0.   -2.08 -2.4 ]
 [-2.36 -2.04 -2.36 -2.44]]
Iteration 3
[[-2.928 -2.872 -2.928 -2.952]
 [-2.784 -2.424 -2.808 -2.928]
 [-2.352  0.    -2.424 -2.872]
 [-2.76  -2.352 -2.784 -2.928]]


## 2. Greedy Policy Extraction

In [5]:
def greedy_policy(V):
    policy = [''] * num_states

    for s in range(num_states):
        if s == terminal_state:
            policy[s] = 'T'
            continue

        best_action = None
        best_value = -np.inf

        for a in actions:
            ns = next_state(s, a)
            value = -1 + gamma * V[ns]

            if value > best_value:
                best_value = value
                best_action = a

        policy[s] = best_action

    return np.array(policy).reshape(grid_size, grid_size)

print('Greedy Policy from Policy Evaluation:')
print(greedy_policy(V_policy))

Greedy Policy from Policy Evaluation:
[['down' 'down' 'down' 'down']
 ['down' 'down' 'down' 'left']
 ['right' 'T' 'left' 'left']
 ['up' 'up' 'left' 'left']]


## 3. Value Iteration

In [6]:
def value_iteration(iterations=3):
    V = -1 * np.ones(num_states)
    V[terminal_state] = 0

    for k in range(iterations):
        new_V = V.copy()

        for s in range(num_states):
            if s == terminal_state:
                continue

            values = []
            for a in actions:
                ns = next_state(s, a)
                values.append(-1 + gamma * V[ns])

            new_V[s] = max(values)

        V = new_V
        print(f'Value Iteration {k+1}')
        print(V.reshape(grid_size, grid_size))

    return V

V_opt = value_iteration(3)

Value Iteration 1
[[-1.8 -1.8 -1.8 -1.8]
 [-1.8 -1.  -1.8 -1.8]
 [-1.   0.  -1.  -1.8]
 [-1.8 -1.  -1.8 -1.8]]
Value Iteration 2
[[-2.44 -1.8  -2.44 -2.44]
 [-1.8  -1.   -1.8  -2.44]
 [-1.    0.   -1.   -1.8 ]
 [-1.8  -1.   -1.8  -2.44]]
Value Iteration 3
[[-2.44  -1.8   -2.44  -2.952]
 [-1.8   -1.    -1.8   -2.44 ]
 [-1.     0.    -1.    -1.8  ]
 [-1.8   -1.    -1.8   -2.44 ]]


## Optimal Policy from Value Iteration

In [7]:
print('Optimal Policy:')
print(greedy_policy(V_opt))

Optimal Policy:
[['down' 'down' 'down' 'down']
 ['down' 'down' 'down' 'down']
 ['right' 'T' 'left' 'left']
 ['up' 'up' 'up' 'up']]
